# Biofabrication — Chapter 4: Interactive Python Exercises (Student Notebook)

**Keep this notebook self-contained.** Each exercise has a short description and a runnable code cell.
Run the **Setup** cell first.

**Topics covered**
- Photocuring dose & exposure
- Layer-based print time (AM)
- SLA exposure budgeting
- Mass of printed implants with infill
- Oxygen diffusion & viable thickness (1D diffusion–reaction, steady-state)
- Effective diffusivity in hydrogels
- Extrusion flow & wall shear stress (Hagen–Poiseuille)
- Composite modulus (Voigt/Reuss bounds)
- (Bonus) 4D bilayer bending (Timoshenko-like approximation)

In [ ]:
# @title Setup (helpers & units)
import math, numpy as np
from textwrap import dedent

# Pretty printer
def hr(title=""):
    line = "="*len(title)
    print(f"\n{line}\n{title}\n{line}")

# Units
mW_per_cm2_to_W_per_m2 = 10.0  # 1 mW/cm^2 = 10 W/m^2
mm_to_m = 1e-3
um_to_m = 1e-6
cm3_to_m3 = 1e-6

## 1) Photocuring Dose → Exposure Time

Given a required dose (mJ/cm²) and intensity (mW/cm²), compute exposure time:
\[
\text{Dose} = \text{Intensity} \times t \quad\Rightarrow\quad t = \frac{\text{Dose}}{\text{Intensity}}
\]

> Try the example (50 mJ/cm² at 10 mW/cm²) and vary intensity.

In [ ]:
# @title Photocuring dose → exposure time
def exposure_time_seconds(required_dose_mJ_cm2: float, intensity_mW_cm2: float) -> float:
    if intensity_mW_cm2 <= 0:
        raise ValueError("Intensity must be > 0")
    return required_dose_mJ_cm2 / intensity_mW_cm2

# Example
t = exposure_time_seconds(50, 10)
hr("Photocuring dose")
print(f"Exposure time: {t:.2f} s  (50 mJ/cm² @ 10 mW/cm²)")

# Explore 5× intensity
t5 = exposure_time_seconds(50, 50)
print(f"At 5× intensity (50 mW/cm²): {t5:.2f} s")

## 2) Layer-by-Layer Build Time

For height \(H\), layer thickness \(\Delta z\), and time per layer \(t_\ell\):
\[
T = \left\lceil \frac{H}{\Delta z} \right\rceil \, t_\ell
\]

In [ ]:
# @title Total build time from layer height & per-layer time
def total_build_time_seconds(height_mm: float, layer_mm: float, time_per_layer_s: float) -> float:
    layers = math.ceil(height_mm / layer_mm)
    return layers * time_per_layer_s

# Example: 12 mm, 0.1 mm layers, 30 s/layer
T = total_build_time_seconds(12.0, 0.1, 30.0)
hr("Layer-by-layer build time")
print(f"Total time: {T/60:.1f} minutes")

## 3) SLA Exposure Budget (Resolution vs Speed)

Compute total exposure for a print:

In [ ]:
# @title SLA exposure time
def sla_exposure_time_seconds(height_mm: float, layer_mm: float, exposure_per_layer_s: float) -> float:
    layers = math.ceil(height_mm / layer_mm)
    return layers * exposure_per_layer_s

# Example: 30 mm, 0.05 mm, 20 s/layer
Te = sla_exposure_time_seconds(30.0, 0.05, 20.0)
hr("SLA exposure budget")
print(f"Total exposure: {Te/3600:.2f} hours")

## 4) Mass of Printed Implant (Infill Fraction)

Mass \(m = V \cdot \rho \cdot f\), where \(f\) is infill fraction.

In [ ]:
# @title Mass from volume, density, infill
def implant_mass_g(volume_cm3: float, density_g_cm3: float, infill_fraction: float) -> float:
    return volume_cm3 * density_g_cm3 * infill_fraction

# Example: 2 cm³ PCL, ρ=1.1 g/cm³, 40% infill
m = implant_mass_g(2.0, 1.1, 0.40)
hr("Implant mass")
print(f"Mass: {m:.3f} g")

## 5) Oxygen Diffusion & Viable Thickness (1D, steady-state)

Steady-state diffusion–reaction on half-thickness \(L\):
\[
D \frac{d^2 c}{dx^2} - k c = 0,\quad c(0)=c_0,\quad \frac{dc}{dx}\big|_{x=L}=0
\]
We search the largest \(L\) such that \(\min c(x) \ge c_{\min}\).

In [ ]:
# @title 1D diffusion–reaction viability thickness (finite difference)
import numpy as np

def viable_half_thickness(c0: float=1.0, cmin: float=0.1,
                          D: float=3e-9,  # m^2/s
                          k: float=0.01   # 1/s effective uptake
                          ) -> float:
    def min_c_for_L(L):
        N = max(50, int(200*L/(200e-6)))
        x = np.linspace(0, L, N+1)
        dx = x[1]-x[0]
        a = D/(dx*dx)
        b = k
        A = np.zeros((N+1, N+1))
        rhs = np.zeros(N+1)
        # Dirichlet at x=0
        A[0,0] = 1.0
        rhs[0] = c0
        # Interior
        for i in range(1, N):
            A[i,i-1] = a
            A[i,i]   = -2*a - b
            A[i,i+1] = a
        # Neumann at x=L
        A[N,N] = 1.0
        A[N,N-1] = -1.0
        sol = np.linalg.solve(A, rhs)
        return sol.min()

    lo, hi = 10e-6, 2e-3
    for _ in range(30):
        mid = 0.5*(lo+hi)
        if min_c_for_L(mid) >= cmin:
            lo = mid
        else:
            hi = mid
    return lo

hr("Viable thickness (diffusion–reaction)")
L = viable_half_thickness(c0=1.0, cmin=0.1, D=3e-9, k=0.01)
print(f"Max half-thickness ≈ {L*1e6:.0f} µm  → full thickness ≈ {2*L*1e6:.0f} µm")

## 6) Effective Diffusivity in Porous Hydrogels

\[
D_e = D_0 \frac{\varepsilon}{\tau}
\]

In [ ]:
# @title Effective diffusivity De = D0 * ε / τ
def effective_diffusivity(D0: float, porosity: float, tortuosity: float) -> float:
    return D0 * porosity / tortuosity

# Example
D0 = 3e-9; eps, tau = 0.8, 2.0
De = effective_diffusivity(D0, eps, tau)
hr("Effective diffusivity")
print(f"De = {De:.2e} m²/s")
print(f"If τ doubles to {2*tau}, De halves → {effective_diffusivity(D0, eps, 2*tau):.2e} m²/s")

## 7) Extrusion Flow & Wall Shear Stress (Hagen–Poiseuille)

\[
Q = \frac{\pi r^4 P}{8 \eta L}, \quad
\tau_w = \frac{4 \eta Q}{\pi r^3}
\]

In [ ]:
# @title Extrusion flow and wall shear stress
def extrusion_Q_and_tau(radius_um: float, length_mm: float,
                        viscosity_Pa_s: float, pressure_kPa: float):
    r = radius_um * um_to_m
    L = length_mm * mm_to_m
    P = pressure_kPa * 1e3
    Q = math.pi * r**4 * P / (8.0 * viscosity_Pa_s * L)  # m^3/s
    tau = 4.0 * viscosity_Pa_s * Q / (math.pi * r**3)    # Pa
    return Q, tau

# Example: r=150 µm, L=5 mm, η=0.1 Pa·s, P=150 kPa
Q, tau = extrusion_Q_and_tau(150.0, 5.0, 0.1, 150.0)
hr("Extrusion & shear")
print(f"Q = {Q*1e9:.2f} nL/s")
print(f"Wall shear τ = {tau:.1f} Pa  (≤ ~200 Pa viability threshold?)")

## 8) Composite Modulus (Voigt/Reuss Bounds)

Upper (iso-strain) and lower (iso-stress) bounds:
\[
E_V = \sum v_i E_i, \qquad
E_R = \left(\sum \frac{v_i}{E_i}\right)^{-1}
\]

In [ ]:
# @title Composite modulus (Voigt/Reuss bounds)
def rule_of_mixtures_voigt(volumes, moduli):
    return sum(v*E for v,E in zip(volumes, moduli))

def rule_of_mixtures_reuss(volumes, moduli):
    return 1.0 / sum(v/E for v,E in zip(volumes, moduli))

# Example: 60% PCL (300 MPa), 40% hydrogel (0.5 MPa)
v = [0.60, 0.40]
E = [300e6, 0.5e6]  # Pa
E_voigt = rule_of_mixtures_voigt(v,E)
E_reuss = rule_of_mixtures_reuss(v,E)
hr("Composite modulus")
print(f"Voigt (iso-strain)  upper bound: {E_voigt/1e6:.2f} MPa")
print(f"Reuss (iso-stress) lower bound: {E_reuss/1e6:.4f} MPa")

## 9) (Bonus) 4D Thermoresponsive Bilayer Bending

Simplified Timoshenko-like curvature for mismatch strain \(\epsilon = \Delta \alpha \Delta T\).

In [ ]:
# @title Bilayer curvature (Timoshenko-like)
def bilayer_curvature_timoshenko(E1, E2, t1, t2, delta_alpha, delta_T):
    n = E2/E1
    m = t2/t1
    epsilon = delta_alpha * delta_T
    kappa = (6*epsilon) / (t1*(1 + 4*m + 6*m**2 + 4*m**3 + m**4) * (1 + n*m))
    return kappa  # 1/m

# Example parameters (students should vary):
E1, E2 = 10e3, 50e3   # Pa
t1, t2 = 0.5e-3, 0.5e-3  # m
delta_alpha = 2e-3       # per °C (effective swell/thermal coefficient difference)
dT = 12.0                # °C (25→37)
hr("4D bilayer curvature")
kappa = bilayer_curvature_timoshenko(E1,E2,t1,t2,delta_alpha,dT)
R = 1.0/kappa if kappa != 0 else float("inf")
print(f"Curvature κ ≈ {kappa:.2f} 1/m  → radius R ≈ {R:.2f} m")